In [1]:
import json
import numpy as np # Для работы с данными 

from tensorflow.keras import utils # Для работы с категориальными данными
from tensorflow.keras.preprocessing.text import Tokenizer # Методы для работы с текстами

In [2]:
dataset_dir = '../../Satellites'

Все тексты true и false объединены в два больших текста;
Датасет - список из двух элементов-строк

In [3]:
# создаём два списка, в пустые элементы которых будем добавлять тексты
# в первом элементе будет храниться текст с ответом False, во втором - True
textClasses = ['', '']


# открываем файл с id эссе и ответами
with open(dataset_dir + '/train/train_standart.json', 'r') as f_list:
  data = json.load(f_list)

  # проходимся по каждому "блоку" с эссе
  for i in range(len(data)):
    elem = data[i]

    with open(dataset_dir + f'/train/essays/{elem["id"]}.json', 'r') as essay:
      file = json.load(essay)
      text = file['text']
      if elem['answer'] == False:
        textClasses[0] += text
        textClasses[0] += '#'
      else:
        textClasses[1] += text
        textClasses[1] += '#'

In [4]:
texts_false = textClasses[0].split("#")
texts_true = textClasses[1].split("#")
border_false = len(texts_false)//10
border_true = len(texts_true)//10

testText = []
testText.append(' '.join(texts_false[:border_false]))
testText.append(' '.join(texts_true[:border_true]))

trainText = []
trainText.append(' '.join(texts_false[border_false:]))
trainText.append(' '.join(texts_true[border_true:]))

In [5]:
numWords = 5000 # Количество слов/индексов, которое мы будем учитывать при обучении

# настраиваем токенайзер, все слова приводятся в нижний регистр, специальные символы опускаются
tokenizer = Tokenizer(num_words=numWords,
                      filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff',
                      lower=True,
                      split=' ',
                      oov_token='unknown',
                      char_level=False)

tokenizer.fit_on_texts(trainText) # словарь частотности
items = list(tokenizer.word_index.items()) # индексы слов

In [6]:
print("Самые часто встречающиеся слова:")
print(items[:10])
print()
print("Самые редко встречающиеся слова:")
print(items[-10:])
print()
print("Размер словаря:", len(items))

Самые часто встречающиеся слова:
[('unknown', 1), ('в', 2), ('и', 3), ('на', 4), ('что', 5), ('не', 6), ('с', 7), ('к', 8), ('как', 9), ('а', 10)]

Самые редко встречающиеся слова:
[('дозволено', 49642), ('мешавшей', 49643), ('старухину', 49644), ('жуткого', 49645), ('сломает', 49646), ('восставала', 49647), ('пришедший', 49648), ('пролитию', 49649), ('евангелие', 49650), ('несущей', 49651)]

Размер словаря: 49651


In [7]:
# Преобразовываем текст в последовательность индексов согласно частотному словарю
trainWordIndexes = tokenizer.texts_to_sequences(trainText)
testWordIndexes = tokenizer.texts_to_sequences(testText)

In [8]:
# классы, на которые делятся тексты
labels = ['false', 'true']
labelsNum = len(labels)

In [9]:
print("Статистика по обучающим текстам:")

symbolsTrainText = 0
wordsTrainText = 0

for i in range(labelsNum):
  print(labels[i], " "*(10-len(labels[i])), len(trainText[i]), "символов, ", len(trainWordIndexes[i]), "слов")
  symbolsTrainText += len(trainText[i])
  wordsTrainText += len(trainWordIndexes[i])

print('----')
print("В сумме ", symbolsTrainText, " символов, ", wordsTrainText, " слов \n")
print()
print("Статистика по тестовым текстам:")

symbolsTestText = 0
wordsTestText = 0

for i in range(labelsNum):
  print(labels[i], ' '*(10-len(labels[i])), len(testText[i]), "символов, ", len(testWordIndexes[i]), "слов")
  symbolsTestText += len(testText[i])
  wordsTestText += len(testWordIndexes[i])
print('----')
print("В сумме ", symbolsTestText, " символов, ", wordsTestText, " слов")


Статистика по обучающим текстам:
false       2339742 символов,  321705 слов
true        596130 символов,  82719 слов
----
В сумме  2935872  символов,  404424  слов 


Статистика по тестовым текстам:
false       247810 символов,  35057 слов
true        58625 символов,  8397 слов
----
В сумме  306435  символов,  43454  слов


In [10]:
def getSetFromIndexes(wordIndexes, xLen, step):
  xText = []
  wordsLen = len(wordIndexes) # Считаем количество слов
  index = 0 # Задаем начальный индекс 

  while (index + xLen <= wordsLen): # Идём по всей длине вектора индексов
    xText.append(wordIndexes[index:index+xLen]) # "Откусываем" векторы длины xLen
    index += step # Смещаемся вперёд на step
    
  return xText


# Формирование обучающей и проверочной выборки для каждого класса
# wordIndexes - массив индексов
# xLen - размер окна
# step - шаг окна

def createSetsMultiClasses(wordIndexes, xLen, step): # Функция принимает последовательность индексов, размер окна, шаг окна
  nClasses = len(wordIndexes) # Количество классов
  classesXSamples = []        # Здесь будет список размером "кол-во классов*кол-во окон в тексте*длину окна"
  for wI in wordIndexes:      # Для каждого текста выборки из последовательности индексов
    classesXSamples.append(getSetFromIndexes(wI, xLen, step))

  # Формируем один общий xSamples
  xSamples = []
  ySamples = []
  
  for t in range(nClasses):
    xT = classesXSamples[t]
    for i in range(len(xT)): # Перебираем каждое окно определенного класса
      xSamples.append(xT[i]) # Добавляем в общий список выборки
      ySamples.append(utils.to_categorical(t, nClasses)) # Добавляем соответствующий вектор класса

  xSamples = np.array(xSamples)
  ySamples = np.array(ySamples)

  return (xSamples, ySamples)

In [13]:
# ЭТИ ПАРАМЕТРЫ МОЖНО И НУЖНО МЕНЯТЬ
# Задаём базовые параметры
xLen = 50 # Размер окна (количество слов в векторе)
step = 6 # Шаг разбиения текста на векторы

In [14]:
xTrainId, yTrain = createSetsMultiClasses(trainWordIndexes, xLen, step)
xTestId, yTest = createSetsMultiClasses(testWordIndexes, xLen, xLen)

# для каждого "окна" класса мы в соответствие ставим значение true или false => поэтому длины x и y одинаковые
print("Размерности тренировочного набора")
print(xTrainId.shape)
print(yTrain.shape)
print()
print("Размерности тестового набора")
print(xTestId.shape)
print(yTest.shape)

Размерности тренировочного набора
(67389, 50)
(67389, 2)

Размерности тестового набора
(868, 50)
(868, 2)


In [15]:
print(yTrain[0])

[1. 0.]


In [16]:
print(xTrainId[0])

[   57     6   387  8522    99 16385   303   642     3     9   564  1405
     7   278    17     2  6448   431   295    14   169  2296  8523   126
     8     1 10164     1     1     1     3     1     1     6  1607    61
  3110 16386  6449     3  5198     2  7377  1067  5783    23   109    60
    81   162]


In [17]:
# Для модели BOW - преобразовываем в Bag Of Words
xTrainBOW = tokenizer.sequences_to_matrix(xTrainId.tolist())
xTestBOW  = tokenizer.sequences_to_matrix(xTestId.tolist())

# Для остальных моделей - НЕ преобразовываем в Bag Of Words

xTrain = xTrainId
xTest = xTestId

print("Размерность обучающей выборки")
print(xTrain.shape)
print(xTrain[0][0:100]) # Фрагмент набора слов 

print()

print("Размерность тестовой выборки")
print(xTest.shape)
print(xTest[0][0:100]) # Фрагмент набора слов

Размерность обучающей выборки
(67389, 50)
[   57     6   387  8522    99 16385   303   642     3     9   564  1405
     7   278    17     2  6448   431   295    14   169  2296  8523   126
     8     1 10164     1     1     1     3     1     1     6  1607    61
  3110 16386  6449     3  5198     2  7377  1067  5783    23   109    60
    81   162]

Размерность тестовой выборки
(868, 50)
[  319   699    35     1  4620     5     2   509    22    57   133   171
   220 14594     1     1  1667   915     8 13060   100   508  3445  7852
    21     1   447    48     1  9296     1    99     1   958  5859    33
   388  1082  4865     1  1881     5     8   811  1632   162   462  6219
     3     1]


In [19]:
np.save('datasets/xTrainBOW.npy', xTrainBOW)
np.save('datasets/xTrain.npy', xTrain)
np.save('datasets/xTestBOW.npy', xTestBOW)
np.save('datasets/xTest.npy', xTest)
np.save('datasets/yTrain.npy', yTrain)
np.save('datasets/yTest.npy', yTest)